In [2]:
# output 39195 hw entries and their inflections and subhw
# format: word (hw, ifl, subhw);hw
# sort by word and make every line unique
# 2015-11-28

# manually delete: asas

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\taglist\\mwl77Entryraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)

# total number of head words
totalhw = 0
totalcnt = 0

for result in results:
    
    word = result.get("hw")    
    if word:
        #print word
        fdict.write(word + ';' + word + '\n')
        totalhw += 1
    else:
        print "no head word"
        continue

    # get items of result
    infls = result.get("infl")
    subhws = result.get("subhw")     
    
    if infls:
        for infl in infls:
            fdict.write(infl + ';' + word + '\n')
            totalcnt += 1   
            
    if subhws:
        for subhw in subhws:
            # check if subhw is a real sub hw of word by comparing first letter 77561 -> 77256
            # no need again because the work has been done in db buidling
            #if subhw[0] != word[0]: continue            
            fdict.write(subhw + ';' + word + '\n')
            totalcnt += 1        

    
fdict.close()
print "total head words: ", totalhw
print "total words: ", totalcnt+totalhw


# sort out by head words
with codecs.open('E:\\1Now\\taglist\\mwl77Entryraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
lines = sorted(set(lines)) # 148596 -> 77256 (before compare subhw and word: 148930 -> 77561)
#lines = sorted(lines)

fd = codecs.open('E:\\1Now\\taglist\\mwl77Entry.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print cnt

total head words:  39195
total words:  148930
77561


In [1]:
# output 39195 hw entries and their summary (subhw, prn, idpvs, tags)
# 2015-11-27

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\taglist\\mwl39195sumraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "subhw":1, "prn":1, "idpvs":1, "tag":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)

# total number of head words
totalhw = 0

for result in results:
    
    word = result.get("hw")    
    if word:
        #print word
        fdict.write(word)
        totalhw += 1
    else:
        print "no head word"
        continue

    # get items of result
    prns = result.get("prn")
    subhws = result.get("subhw")
    idpvs = result.get("idpvs")  
    tags = result.get('tag')
    
    if subhws:
        cnt = 1
        for subhw in subhws:
            if cnt == 1: fdict.write(' (' + subhw)                
            else: fdict.write(' ' + subhw)
            cnt += 1        
        fdict.write(')')
    
    if prns:
        cnt = 1
        for prn in prns:                 
            fdict.write(' /' + prn + '/')
            cnt += 1

    if idpvs:
        cnt = 1
        for idpv in idpvs:
            if cnt == 1: fdict.write('  [' + idpv)
            else: fdict.write(', ' + idpv)
            cnt += 1
        fdict.write(']')
            
    if tags:
        cnt = 1
        for tag in tags:
            if cnt == 1: fdict.write('  {' + tag)
            else: fdict.write(' ' + tag)
            cnt += 1
        fdict.write('}')

    fdict.write('\n')    
    
fdict.close()
print "total head words: ", totalhw


# sort out by head words
with codecs.open('E:\\1Now\\taglist\\mwl39195sumraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
# lines = sorted(set(lines)) #39195 -> 34398
lines = sorted(lines)

fd = codecs.open('E:\\1Now\\taglist\\mwl39195sum.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print cnt

total head words:  39195
39195


In [10]:
# get duplicates
l = [1,2,3,4,4,5,5,6,1]
set([x for x in l if l.count(x) > 1])

{1, 4, 5}

In [9]:
# check word lists only for building mwl mongodb database
# these word lists include (number in parentheses are the number of files)
#   * words of 18 topics from mwl website (18)
#   * 3140 core words from mwl website (1)
#   * words of "The Spelling of Different Sounds in English" (1)
#   * words of word roots, prefixes, suffixes (3)
#   sub-total: 23
#
#   * words of mwl single images (1)
#   * words of mwl group images (106)
#   sub-total: 107
#
#   * homographs (1)
#   * 3787 core words I collect (1)
#   sub-total: 2
# 
#   Total: 131



import codecs
import re

# word list I want to check
# with codecs.open("E:\\1Now\\taglist\\core31.txt", 'r', 'utf-8') as f:
with codecs.open("E:\\1Now\\taglist\\wordRoot.txt", 'r', 'utf-8') as f:
    txtraw = f.read().splitlines()
    f.close()
print 'raw file word number: ', len(txtraw)
txtfine = sorted(set(txtraw))
print 'sort unique word number: ', len(txtfine)

print 'duplicates: ', set([x for x in txtraw if txtraw.count(x) > 1])
    
# mwl77Entry.txt as the standard
# with codecs.open("E:\\1Now\\taglist\\mwl77Entry.txt", 'r', 'utf-8') as f:
with codecs.open("E:\\1Now\\taglist\\mwaled77255entry.txt", 'r', 'utf-8') as f:    
    mwlines = f.read().splitlines()
    f.close()
    
mw77s = []
for line in mwlines:
    entry = {}
    entry['hw'] = line.split(';')[1]
    entry['subhw'] = line.split(';')[0]
#     if entry['hw'] == "they're": print entry
    mw77s.append(entry)
    
    # add a duplicate but first word of subhw is uppercase
    entry = {}
    entry['hw'] = line.split(';')[1]
    entry['subhw'] = line.split(';')[0].capitalize() 
#     if entry['hw'] == "they're": print entry
    mw77s.append(entry)    


txtinmw77 = []
txtnotinmw77 = []
for word in txtraw:
    wordflag = 0
    for item in mw77s:
        if word == item['subhw']:            
            if word != item['hw']: print word, '--hw is--', item['hw']
            txtinmw77.append(item['hw'])
            wordflag = 1            
#             break
    if wordflag == 0: txtnotinmw77.append(word)
# txtinmw77 = sorted(set(txtinmw77))

print '\n---in mw77 words:', len(txtinmw77)
# fout = codecs.open("E:\\1Now\\mae\\article\\outwordlist.txt", 'w', 'utf-8')
# fout.write('---in mw77 words: ' + str(len(txtinmw77)) + '\n')


print '\n---not in mw77:', len(txtnotinmw77)
# fout.write('\n\n---not in mw77: ' + str(len(txtnotinmw77)) + '\n')
for x in txtnotinmw77:
    print x
#     fout.write(x + '\n')

# fout.close()

raw file word number:  323
sort unique word number:  321
duplicates:  set([u'asteroid', u'millennium'])
aerodynamic --hw is-- aerodynamics
disarming --hw is-- disarm
automated --hw is-- automate
exorcist --hw is-- exorcise
feminist --hw is-- feminism
affluence --hw is-- affluent
gradually --hw is-- gradual
homogenized --hw is-- homogenize
dehydrated --hw is-- dehydrate
luminescent --hw is-- luminescence
isometric --hw is-- isometrics
neurologist --hw is-- neurology
annulment --hw is-- annul
technophobia --hw is-- technophobe
populated --hw is-- populate

---in mw77 words: 323

---not in mw77: 2
clarion
telephoto
